# Calculating Ground Motion Intensity Measures

The SMTK contains two modules for the characterisation of ground motion:

1) smtk.response_spectrum

This module contains methods for calculation of the response of a set of single degree-of-freedom (SDOF) oscillators using an input time series. Two methods are currently supported:

    i) Newmark-Beta
    
    ii) Nigam & Jennings (1969) {Preferred}

The module also includes functions for plotting the response spectra and time series

2) smtk.intensity_measures 

This module contains a set of functions for deriving different intensity measures from a strong motion record

i) get_peak_measures(...) - returns PGA, PGV and PGD

ii) get_response_spectrum(...) - returns the response spectrum

iii) get_response_spectrum_pair(...) - returns a response spectrum pair

iv) geometric_mean_spectrum(...) - returns the geometric mean of a pair of records

v) arithmetic_mean_spectrum(...) - returns the arithmetic mean of a pair of records

vi) geometric_mean_spectrum(...) - returns the envelope spectrum of a pair of records

vii) larger_pga(...) - Returns the spectrum with the larger PGA

viii) rotate_horizontal(...) - rotates a record pair through angle theta

ix) gmrotdpp(...) - Returns the rotationally-dependent geometric fractile (pp) of a pair of records

x) gmrotipp(...) - Returns the rotationally-independent geometric fractile (pp) of a pair of records


### Example Usage of the Response Spectrum

In [ ]:
%load_ext autoreload
%autoreload 2
import warnings; warnings.filterwarnings("ignore")

In [ ]:
# Import modules
%matplotlib inline
import numpy as np  # Numerical Python package
import matplotlib.pyplot as plt # Python plotting package
# Import
import smtk.response_spectrum as rsp # Response Spectra tools
import smtk.intensity_measures as ims # Intensity Measure Tools


periods = np.array([0.01, 0.02, 0.03, 0.04, 0.05, 0.075, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19,
                    0.20, 0.22, 0.24, 0.26, 0.28, 0.30, 0.32, 0.34, 0.36, 0.38, 0.40, 0.42, 0.44, 0.46, 0.48, 0.5, 
                    0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 
                    1.9, 2.0, 2.2, 2.4, 2.6, 2.8, 3.0, 3.2, 3.4, 3.6, 3.8, 4.0, 4.2, 4.4, 4.6, 4.8, 5.0, 5.5, 6.0, 
                    6.5, 7.0,7.5, 8.0, 8.5, 9.0, 9.5, 10.0], dtype=float)
number_periods = len(periods)

In [ ]:
# Load record pair from files
x_record = np.genfromtxt("data/sm_record_x.txt")
y_record = np.genfromtxt("data/sm_record_y.txt")

x_time_step = 0.002 # Record sampled at 0.002 s  
y_time_step = 0.002

### Get Response Spectrum - Nigam & Jennings

In [ ]:
# Create an instance of the Newmark-Beta class
nigam_jennings = rsp.NigamJennings(x_record, x_time_step, periods, damping=0.05, units="cm/s/s")
sax, time_series, acc, vel, dis = nigam_jennings()

# Plot Response Spectrum
rsp.plot_response_spectra(sax, axis_type="semilogx",
                          filename="images/response_nigam_jennings.pdf",
                          filetype="pdf")

### Plot Time Series

In [ ]:
rsp.plot_time_series(time_series["Acceleration"],
                     x_time_step,
                     time_series["Velocity"],
                     time_series["Displacement"])

### Intensity Measures

#### Get PGA, PGV and PGD

In [ ]:
pga_x, pgv_x, pgd_x, _, _ = ims.get_peak_measures(0.002, x_record, True, True)
print("PGA = %10.7f cm/s/s,  PGV = %10.7f cm/s,   PGD = %10.7f cm" % (pga_x, pgv_x, pgd_x))
pga_y, pgv_y, pgd_y, _, _ = ims.get_peak_measures(0.002, y_record, True, True)
print("PGA = %10.7f cm/s/s,  PGV = %10.7f cm/s,   PGD = %10.7f cm" % (pga_y, pgv_y, pgd_y))

#### Get Durations: Bracketed, Uniform, Significant

In [ ]:
print("Bracketed Duration (> 5 cm/s/s) = %9.7f s" % ims.get_bracketed_duration(x_record, x_time_step, 5.0))
print("Uniform Duration (> 5 cm/s/s) = %9.7f s" % ims.get_uniform_duration(x_record, x_time_step, 5.0))
print("Significant Duration (5 - 95 Arias ) = %9.7f s" % ims.get_significant_duration(x_record, x_time_step, 0.05, 0.95))

#### Get Arias Intensity, CAV, CAV5 and rms acceleration

In [ ]:
print("Arias Intensity = %12.7f cm-s" % ims.get_arias_intensity(x_record, x_time_step))
print("Arias Intensity (5 - 95) = %12.7f cm-s" % ims.get_arias_intensity(x_record, x_time_step, 0.05, 0.95))
print("CAV = %12.7f cm-s" % ims.get_cav(x_record, x_time_step))
print("CAV5 = %12.7f cm-s" % ims.get_cav(x_record, x_time_step, threshold=5.0))
print("Arms = %12.7f cm-s" % ims.get_arms(x_record, x_time_step))

##### Spectrum Intensities: Housner Intensity, Acceleration Spectrum Intensity

In [ ]:
# Get response spectrum
sax = ims.get_response_spectrum(x_record, x_time_step, periods)[0]
print("Velocity Spectrum Intensity (cm/s/s) = %12.7f" % ims.get_response_spectrum_intensity(sax))
print("Acceleration Spectrum Intensity (cm-s) = %12.7f" % ims.get_acceleration_spectrum_intensity(sax))


#### Get the response spectrum pair from two records

In [ ]:
sax, say = ims.get_response_spectrum_pair(x_record, x_time_step,
                                          y_record, y_time_step,
                                          periods,
                                          damping=0.05,
                                          units="cm/s/s",
                                          method="Nigam-Jennings")


#### Get Geometric Mean Spectrum

In [ ]:
sa_gm = ims.geometric_mean_spectrum(sax, say)
rsp.plot_response_spectra(sa_gm, "semilogx", filename="images/geometric_mean_spectrum.pdf", filetype="pdf")

#### Get Envelope Spectrum

In [ ]:
sa_env = ims.envelope_spectrum(sax, say)
rsp.plot_response_spectra(sa_env, "semilogx", filename="images/envelope_spectrum.pdf", filetype="pdf")

### Rotationally Dependent and Independent IMs

#### GMRotD50 and GMRotI50

In [ ]:
gmrotd50 = ims.gmrotdpp(x_record, x_time_step, y_record, y_time_step, periods, percentile=50.0,
                                               damping=0.05, units="cm/s/s")
gmroti50 = ims.gmrotipp(x_record, x_time_step, y_record, y_time_step, periods, percentile=50.0,
                                               damping=0.05, units="cm/s/s")

In [ ]:
# Plot all of the rotational angles!
plt.figure(figsize=(8, 6))
for row in gmrotd50["GeoMeanPerAngle"]:
    plt.semilogx(periods, row, "-", color="LightGray")
plt.semilogx(periods, gmrotd50["GMRotDpp"], 'b-', linewidth=2, label="GMRotD50")
plt.semilogx(periods, gmroti50["Pseudo-Acceleration"], 'r-', linewidth=2, label="GMRotI50")
plt.xlabel("Period (s)", fontsize=18)
plt.ylabel("Acceleration (cm/s/s)", fontsize=18)
plt.legend(loc=0)
plt.savefig("images/rotational_spectra.pdf", dpi=300, format="pdf")
